In [ ]:
import geemap
import ee

Map = geemap.Map()

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile({'q':"mimeType='image/tiff'"}).GetList()
file_list

In [ ]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)
file_list = drive.ListFile({'q':"mimeType='image/tiff'"}).GetList()
for file in file_list:
    out_file = path + file['title']
    file.GetContentFile(out_file)
    print(out_file+'下载成功！！')
    file.Delete()

In [ ]:
roi = geemap.geojson_to_ee(r'E:/Data/GIS/JX.json')
modis = ee.ImageCollection("MODIS/006/MOD09Q1").filterDate('2017', '2018')
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI')\
            .filterDate('2017', '2018')
modis_names = modis.aggregate_array('system:id').getInfo()
landsat_names = landsat8.aggregate_array('system:id').getInfo()

In [ ]:
def clip(img):
    return img.clip(roi.geometry())
def modis_NDVI(img):
    img1 = img.clip(roi.geometry())
    img2 = img1.normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])
    return img2
def export_image(i):
    modis_name = modis_names[i]
    landsat_name = landsat_names[i]
    modis_img = modis_NDVI(ee.Image(modis_name))
    landsat_img = clip(ee.Image(landsat_name))
    NDVI_img = landsat_img.unmask(modis_img)
    f_name = 'NDVI_'+modis_name.split('/')[-1]
    geemap.ee_export_image_to_drive(NDVI_img, description=f_name,
                                    folder='Image', scale=30)
    return f_name+'.tif'

In [ ]:
import time
path = r'E:/Data/NDVI_JX/'
# title = export_image(6)
for i in range(13, len(modis_names)+1):
        out_file = path + title
        file_list = []
        while(len(file_list) == 0):
            file_list = drive.ListFile({'q':"title='{0}'".format(title)}).GetList()
        if i != len(modis_names):
            title = export_image(i)
        file = file_list[0]
        file.GetContentFile(out_file)
        print(out_file+'下载成功！！')
        file.Delete()
        

In [ ]:
title

In [ ]:
import winsound
duration = 1000  # millisecond
freq = 440  # Hz
winsound.Beep(freq, duration)